# Local TIG Algorithm Tester - single algorithm

Tests a selected algorithm from currently pushed algorithms or for testing the performance of your own algorithm. Settings:
- Challenge name
- Algorithm name (path)
- Number of nonces/instances to test (Default 1000 nonces)
- Difficulty selection
- Number of cores used to test (Default 1 core)

In [ ]:
#importing modules

import ipywidgets as widgets
import matplotlib.pyplot as plt
import os
import sys
import requests
import multiprocessing
import time
from tools import run_test, check_algorithm_existence, build_worker, dump_difficulty, get_wasm_blob

## Pulling Round Data
If an error shows, run again as sometimes the server is not as cooperative until "Complete" is printed

In [ ]:
# pulls data from the current block and round
url = "https://mainnet-api.tig.foundation/"
block_data = requests.get(f"{url}get-block").json()
latest_block = block_data["block"]["id"]

challenge_data = requests.get(f"{url}get-challenges?block_id={latest_block}").json()
algorithm_data = requests.get(f"{url}get-algorithms?block_id={latest_block}").json()

challenge_id_dict = {challenge["id"]:[challenge["details"]["name"]] for challenge in challenge_data["challenges"]}
algorithm_id_dict = {algorithm["id"]:algorithm["details"]["name"] for algorithm in algorithm_data["algorithms"] if algorithm["state"]["round_pushed"]}
[challenge_id_dict[algorithm["id"][:4]].append(algorithm["id"]) for algorithm in algorithm_data["algorithms"] if algorithm["state"]["round_pushed"]]

print("Complete")

## Algorithm and settings selection

Add your algorithm's compiled .wasm file into the /wasm folder, and specify the name (e.g. for a file named knap_speed.wasm, enter: knap_speed)

**Make sure you select the right challenge for the algorithm you want to test**

In [ ]:
challenge_name = widgets.interactive(lambda Challenge:Challenge, Challenge=["satisfiability", "knapsack", "vehicle_routing"])
display(challenge_name)

In [ ]:
#Run this line to see all available algorithms for the challenge selected above
id = None
for c_id, c_info in challenge_id_dict.items():
    if c_info[0] == challenge_name.result:
        id = c_id

for algo in challenge_id_dict[id][1:]:
    print(algorithm_id_dict[algo])

In [ ]:
algorithm_name = widgets.interactive(lambda Name:Name, Name="Algorithm name")
display(algorithm_name)

In [ ]:
#Run this box to check that your algorithm exists
if check_algorithm_existence(algorithm_name.result, algorithm_id_dict=algorithm_id_dict):
    print("Your algorithm exists")
else:
    print("Your algorithm does not exist, please check again")

Enter the integer of the difficulty of each parameter e.g. for [20, 250], enter 20 and 250

* [Difficulty for Satisfiability] Recommend [50, 300] for initial tests
* [Difficulty for Vehicle Routing] Recommend [40, 250] for initial tests
* [Difficulty for Knapsack] Recommend [50, 10] for initial tests


In [ ]:
param_1 = widgets.interactive(lambda param_1:param_1, param_1="0")
param_2 = widgets.interactive(lambda param_2:param_2, param_2="0")

display(param_1)
display(param_2)

Enter the number of instances you would like to test your algorithm for as an integer

In [ ]:
num_nonces = widgets.interactive(lambda instances:instances, instances="1")
display(num_nonces)

Select the number of cores you want to use during the tests (out of your maximum cores)

In [ ]:
num_cores = widgets.interactive(lambda x=1:x, x=(1, multiprocessing.cpu_count(),1))
display(num_cores)

## Testing Algorithm

In [ ]:
#Run this cell until "Complete" is printed out
%env RUSTFLAGS=-Awarnings

build_worker()
dump_difficulty([param_1.result, param_2.result])

if check_algorithm_existence(algorithm_name.result, algorithm_id_dict=algorithm_id_dict):
    os.environ["algorithm_name"] = algorithm_name.result
else:
    sys.exit("Your algorithm does not exist, please re enter an existing algorithm before attempting to test it")

if not os.path.exists(f"./wasm/{algorithm_name.result}.wasm"):
    branch_name = f"{challenge_name.result}/{algorithm_name.result}"
    get_wasm_blob(algorithm_name.result, challenge_name.result)

start_time = time.time()
with multiprocessing.Pool(num_cores.result) as pool:
    results = pool.map(run_test, range(int(num_nonces.result)))
end_time = time.time()

running_time = end_time - start_time
num_solutions = results.count(0)
num_invalid_solutions = results.count(1)
num_errors = results.count(2)

print("Complete")

**Make sure to re-run the test if you have changed any of the setting**

In [ ]:
print(f"Challenge name: {challenge_name.result}, algorithm name: {algorithm_name.result} at difficulty {[param_1.result, param_2.result]}")
print(f"Tested {num_nonces.result} instances in {running_time*1000} ms")
print(f"Found {num_solutions} solutions, received {num_errors} errors and {num_invalid_solutions} invalid solutions")
if num_solutions != 0:
    print(f"Resulting in an avg_time_per_solution of {(running_time / num_solutions)*1000} ms")
else:
    print("No solutions were found")

## Results Bar Chart

Run the cell below to output a simple bar chart

In [ ]:
fig = plt.figure(figsize = (10, 5))
plt.bar(["Instances", "Solutions", "Invalid Solutions", "Errors"],
        [int(num_nonces.result), num_solutions, num_invalid_solutions, num_errors],
        color = "green", width = 0.4)
plt.xlabel("Results")
plt.ylabel("Occurrences")
plt.title(f"Challenge: {challenge_name.result}     Algorithm: {algorithm_name.result}     Difficulty: {[param_1.result, param_2.result]}     Time: {(running_time)*1000:.2f}ms")
plt.show()